# Calculating the similarity matrix and using it to fit an SVM

In [ ]:
import numpy as np
import pandas as pd
import utils
import turicreate as tc
import matplotlib.pyplot as plt

In [ ]:
data = tc.SFrame({
    'x1': [0,-1,0,0,1,-1,1],
    'x2': [0,0,-1,1,0,1,-1],
    'y': [0,0,0,1,1,1,1]
})
data

In [ ]:
utils.plot_points(pd.DataFrame(data['x1','x2']), pd.DataFrame(data['y']))

### Calculating the similarity matrix

In [ ]:
def similarity(x, y):
    return np.exp(-(x[0]-y[0])**2-(x[1]-y[1])**2)

In [ ]:
data_with_similarities = tc.SFrame(data)

for i in range(len(data)):
    column = []
    for j in range(len(data)):
        first = [data[i]['x1'], data[i]['x2']]
        second = [data[j]['x1'], data[j]['x2']]
        column.append(similarity(first, second))
    data_with_similarities = data_with_similarities.add_column(column, 'Sim'+str(i))
data_with_similarities

### Fitting a (linear) SVM to the similarity matrix

In [ ]:
svm = tc.svm_classifier.create(data_with_similarities, target='y')

In [ ]:
svm.coefficients

In [ ]:
coefs = svm.coefficients['value']
coefs

### Plotting the classifier

In [ ]:
def svm_rbf(p):
    similarities = [similarity(p, [row['x1'], row['x2']]) for row in data]
    return np.dot(similarities, [-1,-1,-1,1,1,1,1])

In [ ]:
features = np.array(pd.DataFrame(data['x1','x2']))
labels = np.array(data['y'])

In [ ]:
def plot_model(X, y, model):
    X = np.array(X)
    y = np.array(y)
    plot_step = 0.1
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
    Z = np.array([model(i) for i in np.c_[xx.ravel(), yy.ravel()]])
    Z = Z.reshape(xx.shape)
    plt.contour(xx, yy, Z,colors = 'k',linewidths = 3)
    plt.contourf(xx, yy, Z, colors=['red', 'blue'], alpha=0.2, levels=[-20,0,20])
    utils.plot_points(X, y)
    plt.show()

In [ ]:
plot_model(features, labels, svm_rbf)